In [48]:
#
import os
import copy
import pytz
import sys
import warnings
import matplotlib
from pathlib import Path
import dataclasses
from typing import Dict, Union

#
sys.path.append("./src/")
sys.path.append("./")
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

# FinRL
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

# FinRL-Meta
from stable_baselines3 import A2C
from stable_baselines3 import DDPG
from stable_baselines3 import PPO
from stable_baselines3 import SAC
from stable_baselines3 import TD3

#
import yfinance as yf

In [49]:
def config():
    #
    warnings.filterwarnings("ignore", category=UserWarning)  # TODO: zipline problem
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=RuntimeWarning)

    #
    matplotlib.use("Agg")


config()

In [50]:
#
import numpy as np
import pandas as pd
import seaborn as sns
import tqdm

#
from common.utils import now_time
from configuration.settings import ProjectDir, ExperimentDir
from rl.data.CompanyInfo import CompanyInfo
from rl.experiments._1_same_bigger_data_fundamental.train import (
    Program,
    get_dataset,
    dataset_name,
    get_env_kwargs,
    CustomDRLAgent,
)
from rl.experiments._1_same_bigger_data_fundamental.StockTradingEnv import StockTradingEnv

In [51]:
#
program = Program(
    prj_dir=ProjectDir(root=Path("/Users/zlapik/my-drive-zlapik/0-todo/ai-investing")),
    exp_dir=ExperimentDir(Path(os.getcwd())),
    DEBUG=False,
)
program.dataset = get_dataset(
    pd.read_csv(program.exp_dir.out.datasets.joinpath(f"{dataset_name}.csv"), index_col=0), purpose="test"
)
program.exp_dir.check_and_create_dirs()

In [52]:
from rl.experiments._1_same_bigger_data_fundamental.train import base_cols, data_cols, ratios_cols

print(base_cols)
print(data_cols)
print(ratios_cols)

['date', 'tic']
['open', 'high', 'low', 'close', 'volume']
['operatingProfitMargin', 'netProfitMargin', 'returnOnAssets', 'returnOnEquity', 'currentRatio', 'quickRatio', 'cashRatio', 'inventoryTurnover', 'receivablesTurnover', 'payablesTurnover', 'debtRatio', 'debtEquityRatio', 'priceEarningsRatio', 'priceBookValueRatio', 'dividendYield']


In [53]:
@dataclasses.dataclass
class LearnedAlgorithm:
    algorithm: str
    filename: Path
    learned_algorithm: Union[A2C, PPO, DDPG, A2C, TD3]


def get_algorithm(filename: Path):
    if "a2c" in filename.as_posix():
        return LearnedAlgorithm(algorithm="a2c", filename=filename, learned_algorithm=A2C.load(filename))


learned_algorithms = [get_algorithm(filepath) for filepath in program.exp_dir.out.algorithms.glob("*")]

In [54]:
env_kwargs = get_env_kwargs(program.dataset)
env_gym = StockTradingEnv(df=program.dataset, **env_kwargs)

Stock Dimension: 29, State Space: 494


In [71]:
df_account_value, df_actions = CustomDRLAgent.DRL_prediction(
    model=learned_algorithms[len(learned_algorithms) - 1].learned_algorithm, environment=env_gym
)
# df_account_value = df_account_value.set_index('date')

hit end!


In [72]:
df_account_value

,date,account_value
0,2019-04-12,1.000000e+06
1,2019-04-15,9.994271e+05
2,2019-04-16,1.000304e+06
3,2019-04-17,9.962704e+05
4,2019-04-18,9.988517e+05
...,...,...
923,2022-12-09,1.241070e+06
924,2022-12-12,1.260773e+06
925,2022-12-13,1.278515e+06
926,2022-12-14,1.262068e+06


ValueError: Could not interpret value `x` for parameter `x`

In [79]:
ax = sns.lineplot(data=df_account_value, x="date", y="account_value")
ax

<AxesSubplot: xlabel='date', ylabel='account_value'>

In [75]:
sns.lmplot(data=df_account_value, x="date", y="account_value")

ValueError: could not convert string to float: '2019-04-12'

In [33]:
print(df_account_value)

           date  account_value
0    2019-04-12   1.000000e+06
1    2019-04-15   9.994271e+05
2    2019-04-16   1.000304e+06
3    2019-04-17   9.962704e+05
4    2019-04-18   9.988517e+05
..          ...            ...
923  2022-12-09   1.241070e+06
924  2022-12-12   1.260773e+06
925  2022-12-13   1.278515e+06
926  2022-12-14   1.262068e+06
927  2022-12-15   1.229539e+06

[928 rows x 2 columns]


In [ ]:
perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv(program.exp_dir.out.results.joinpath("perf_stats_all_ppo_" + now_time() + ".csv"))

In [26]:
# Baseline
baseline_df = get_baseline(ticker="^DJI", start=program.dataset["date"].min(), end=program.dataset["date"].max())
stats = backtest_stats(baseline_df, value_col_name="close")

[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (927, 8)
Annual return          0.070772
Cumulative returns     0.286005
Annual volatility      0.232728
Sharpe ratio           0.411239
Calmar ratio           0.190831
Stability              0.561379
Max drawdown          -0.370862
Omega ratio            1.087791
Sortino ratio          0.567932
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.953450
Daily value at risk   -0.028941
dtype: float64


In [21]:
backtest_plot(
    df_account_value,
    baseline_ticker="^DJI",
    baseline_start=program.dataset["date"].min(),
    baseline_end=program.dataset["date"].max(),
)

[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (927, 8)


Start date,2019-04-12
End date,2022-12-15
Total months,44
,Backtest
Annual return,5.772%
Cumulative returns,22.954%
Annual volatility,29.539%
Sharpe ratio,0.34
Calmar ratio,0.13
Stability,0.36
Max drawdown,-44.159%


Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,44.16,2020-02-12,2020-03-20,2021-01-06,236
1,32.13,2021-11-02,2022-09-27,NaT,NaN
2,9.42,2019-07-23,2019-08-14,2019-09-11,37
3,7.11,2019-04-30,2019-06-03,2019-06-20,38
4,6.81,2021-08-27,2021-09-21,2021-10-18,37


Stress Events,mean,min,max
New Normal,0.04%,-15.71%,13.63%
